Στο αρχείο αυτό έχουμε τρέξει το τελευταίο κομμάτι της μεθόδου ARIMA. Για να τρέξει τοποθετήστε στον ίδιο φάκελο τα δεδομένα.

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [ ]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h4>Διάβασμα αρχικών δεδομένων</h4>

In [ ]:
data = pd.read_csv('datapoints.csv', header=0)
data.head()

,Time,pwrA,pwrB,pwrC
0,2021-12-19T12:48:03.379000Z,523.103149,303.693207,163.850006
1,2021-12-19T12:49:03.378000Z,530.507996,298.835236,164.227325
2,2021-12-19T12:50:03.377000Z,522.961670,304.117676,165.689423
3,2021-12-19T12:51:03.377000Z,523.952148,309.400116,164.132996
4,2021-12-19T12:52:03.376000Z,523.103149,298.269257,175.122360


In [ ]:
#κατάλληλη μετατροπή πεδίου Time σε δεδομένα χρόνου
data['Time'] = pd.to_datetime(data['Time'],format="%Y-%m-%dT%H:%M:%S.%f").dt.tz_convert(tz='EET') #αλλαγή του data type σε datetime
print(data['Time'].dtype)
print(data['pwrA'].dtype)

datetime64[ns, EET]
float64


<h4>Παραγωγή τελικού συνόλου δεδομένων</h4>

In [ ]:
actual_data = data.resample('H',on='Time').mean()

In [ ]:
resampled = data.resample('H',on='Time').mean()
resampled

,pwrA,pwrB,pwrC
Time,,,
2021-12-19 14:00:00+02:00,524.132919,306.306908,167.265511
2021-12-19 15:00:00+02:00,525.374743,511.348559,167.484606
2021-12-19 16:00:00+02:00,531.168318,339.355525,167.658640
2021-12-19 17:00:00+02:00,526.637219,519.724122,166.228710
2021-12-19 18:00:00+02:00,522.449141,302.335642,168.717396
...,...,...,...
2022-01-16 10:00:00+02:00,565.430094,368.923144,135.492783
2022-01-16 11:00:00+02:00,561.919671,359.555013,140.070370
2022-01-16 12:00:00+02:00,551.333695,474.682601,134.745857


<h4>Χειρισμός μηδενικών και ακραίων τιμών</h4>

In [ ]:
#6% των παρατηρήσεων είναι αποδεκτό ποσοστό 
#topping των outliers
resampled['pwrA'].loc[(resampled['pwrA']>595)] = 595
resampled['pwrA'].loc[(resampled['pwrA']<525)] = 525

In [ ]:
#0.1% των παρατηρήσεων είναι αποδεκτό ποσοστό 
#topping των outliers
resampled['pwrB'].loc[(resampled['pwrB']>680)] = 680
resampled['pwrB'].loc[(resampled['pwrB']<210)] = 210

In [ ]:
#υπερβολικά υψηλό ποσοστό
#θα ακολουθήσουμε άλλη μέθοδο
lower_limit = resampled['pwrC'].quantile(0.001)
upper_limit = resampled['pwrC'].quantile(0.92)
resampled['pwrC'] = resampled['pwrC'].clip(lower=lower_limit,upper=upper_limit)

#ARIMA

#Ορίζοντας 24#

In [ ]:
import pmdarima as pm

horizon = 24
model = pm.auto_arima(resampled['pwrB'][:-horizon],trace=True ,max_p=20, max_q=20, max_order=36, stepwise=False, suppress_warnings=True, seasonal=False)

 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=8022.379, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=7522.209, Time=0.55 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=7486.631, Time=0.88 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=7487.693, Time=0.89 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=7475.517, Time=1.49 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=7475.954, Time=2.54 sec
 ARIMA(0,1,6)(0,0,0)[0] intercept   : AIC=7476.652, Time=1.38 sec
 ARIMA(0,1,7)(0,0,0)[0] intercept   : AIC=7473.739, Time=1.92 sec
 ARIMA(0,1,8)(0,0,0)[0] intercept   : AIC=7475.739, Time=2.52 sec
 ARIMA(0,1,9)(0,0,0)[0] intercept   : AIC=7477.678, Time=2.92 sec
 ARIMA(0,1,10)(0,0,0)[0] intercept   : AIC=7478.612, Time=3.51 sec
 ARIMA(0,1,11)(0,0,0)[0] intercept   : AIC=7479.414, Time=4.19 sec
 ARIMA(0,1,12)(0,0,0)[0] intercept   : AIC=7481.410, Time=4.32 sec
 ARIMA(0,1,13)(0,0,0)[0] intercept   : AIC=7478.105, Time=5.65 sec
 ARIMA(0,1,14)(0,0,0)[0] intercept   : AIC=7480.094, Time=6.46 sec
 ARIM

In [ ]:
test_preds = model.predict(n_periods=horizon, return_conf_int=False)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

print(100*mean_absolute_percentage_error(actual_data['pwrB'][-horizon:],test_preds))

15.509270961386182


In [ ]:
test_preds

array([405.47012726, 451.51235918, 404.99749265, 401.70172971,
       437.23668262, 403.69465022, 408.26803997, 431.2289214 ,
       398.6908673 , 415.30866373, 423.97052679, 397.00183471,
       420.45962859, 415.97790587, 398.65843843, 422.79137281,
       408.71477871, 402.50806243, 422.21790083, 403.32153366,
       407.15764127, 419.29109099, 400.35843333, 411.35245678])